In [ ]:
import numpy as np
import igl
import meshplot as mp
from scipy.spatial.transform import Rotation
import ipywidgets as iw
import time
from deform_ops_chol import Deformer # or from deform_ops_chol import Deformer 

# Multiresolution mesh editing: 
For this task, you will compute a mesh deformation based on the rotations and translations applied interactively to a subset of its vertices via the mouse. Let $H$ be the set of "handle" vertices that the user can manipulate (or leave fixed). We want to compute a deformation for the remaining vertices, denoted as $R$.

Let $\mathcal{S=(v,f)}$ be our input surface, represented as a triangle mesh. We want to compute a new surface that contains:
- the vertices in $H$ (the handles) translated/rotated using the user-provided transformation $t$, and
- the vertices in $R$ properly deformed using the algorithm described next.

The algorithm is divided in three phases:

1. removing high-frequency details:
`B = deformer_.smooth_mesh(v)`
2. deforming the smooth mesh, and
`B' = deformer_.smooth_mesh(new_pos)`
3. transferring high-frequency details to the deformed surface (encoding details in B in a reference frame, and applying them to B_prime, )

    `coeffs, tangents = deformer_.compute_detail_encoding(B)` 

    `B_prime_dets = deformer_.apply_detail_encoding(B_prime,coeffs, tangents)`

    `S_prime = B_prime + deformer_.details`

My implementation of these operations is contained in  `deform_ops.py`, and a performance-optimized version is contained in `deform_ops_chol.py`. In order to do a deformation, an instance of the deformer must be initialized with the original vertices and faces of the mesh, as well as the indices of the handles. 

## Interactive Version

In [19]:
# v, f = igl.read_triangle_mesh('data/hand.off')
# labels = np.load('data/hand.label.npy').astype(int)

v, f = igl.read_triangle_mesh('data/cactus.off')
labels = np.load('data/cactus.label.npy').astype(int)

# v, f = igl.read_triangle_mesh('data/woody-hi.off')
# labels = np.load('data/woody-hi.label.npy').astype(int)

# v, f = igl.read_triangle_mesh('data/woody-lo.off')
# labels = np.load('data/woody-lo.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()

In [20]:
handle_vertex_positions = v.copy()
pos_f_saver = np.zeros((labels.max() + 1, 6)) # saves transformations in a martix 
handle_indices = np.where(labels!=0)[0]
deformer_ = Deformer(v, f, handle_indices)
def pos_f(s,x,y,z, α, β, γ):
    slices = (labels==s)
    # print('slices', slices)
    r = Rotation.from_euler('xyz', [α, β, γ], degrees=True)
    v_slice = v[slices] + np.array([[x,y,z]])
    center = v_slice.mean(axis=0)
    handle_vertex_positions[slices] = r.apply(v_slice - center) + center
    pos_f_saver[s - 1] = [x,y,z,α,β,γ]
    t0 = time.time()
    v_deformed = pos_f.deformer(handle_vertex_positions)
    p.update_object(vertices = v_deformed)
    t1 = time.time()
    print('FPS', 1/(t1 - t0))
pos_f.deformer = lambda x:x

In [21]:
def widgets_wrapper():
    segment_widget = iw.Dropdown(options=np.arange(labels.max()) + 1)
    translate_widget = {i:iw.FloatSlider(min=-1, max=1, value=0) 
                        for i in 'xyz'}
    rotate_widget = {a:iw.FloatSlider(min=-90, max=90, value=0, step=1) 
                     for a in 'αβγ'}

    def update_seg(*args):
        (translate_widget['x'].value,translate_widget['y'].value,
        translate_widget['z'].value,
        rotate_widget['α'].value,rotate_widget['β'].value,
        rotate_widget['γ'].value) = pos_f_saver[segment_widget.value]
    segment_widget.observe(update_seg, 'value')
    widgets_dict = dict(s=segment_widget)
    widgets_dict.update(translate_widget)
    widgets_dict.update(rotate_widget)
    return widgets_dict

In [22]:
def position_deformer(target_pos):
    '''Fill in this function to change positions'''
    # smooth original mesh (except vertex handles)
    B = deformer_.smooth_mesh(v)
    # smooth the target mesh (except vertex handles)
    B_prime =  deformer_.smooth_mesh(target_pos)
    # encode the details of B into a set of coefficients and tangents
    coeffs, tangents = deformer_.compute_detail_encoding(B)
    # apply the detail encoding to the target mesh
    B_prime_dets = deformer_.apply_detail_encoding(B_prime,coeffs, tangents)
    target_pos = B_prime + B_prime_dets
    return target_pos
pos_f.deformer = position_deformer

In [23]:
p = mp.plot(handle_vertex_positions, f, c=labels)
iw.interact(pos_f,
            **widgets_wrapper())

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0736866…

interactive(children=(Dropdown(description='s', options=(1, 2, 3, 4), value=1), FloatSlider(value=0.0, descrip…

<function __main__.pos_f(s, x, y, z, α, β, γ)>

## Non-interactive version + Methodology Explanation

#### Step 1: Smooth base surface (removal of high-frequency details) 

This is the `B = deformer_.smooth_mesh(v)`

Note that `smooth_mesh`, takes a value for the `new` positions of the vertices. Since smoothing the base surface does not have a new position of the vertices, we put in the original vertices as the new ones. 

We remove the high-frequency details from the vertices $R$ in $\mathcal{S}$ by minimizing the thin-plate energy, which involves solving a bi-Laplacian system arising from the quadratic energy minimization:
<img align="center" width="200" src="https://i.imgur.com/6IRzdBj.png">

where $o_H$ are the handle $H$'s vertex positions, $L_\omega$ is the cotan Laplacian of $\mathcal{S}$, and $M$ is the mass matrix of $\mathcal{S}$.

Notice that $L_\omega$ is the symmetric matrix consisting of the cotangent weights ONLY (without the division by Voronoi areas). In other words, it evaluates an "integrated" Laplacian rather than an "averaged" laplacian when applied to a vector of vertices. The inverse mass matrix appearing in the formula above then applies the appropriate rescaling so that the laplacian operator can be applied again (i.e., so that the Laplacian value computed at each vertex can be interpreted as a piecewise linear scalar field whose Laplacian can be computed).
This optimization will produce a mesh similar to the one in Figure 2. Note that the part of the surface that we want to deform is now free of high-frequency details. We call this mesh $\mathcal{B}$.

*Relevant `scipy` functions:* `scipy.sparse.csc_matrix`, `scipy.sparse.diags`, 

<!-- # Laplacian smoothing = doing explicit smoothung  -->
<!-- -- compute new pos of vertex as pos of vertex + displacement * laplacian  -->

In [24]:
# load the original mesh and get the handle indices
v, f = igl.read_triangle_mesh('data/woody-hi.off')
labels = np.load('data/woody-hi.label.npy').astype(int)
v -= v.min(axis=0)
v /= v.max()
handle_indices = np.where(labels!=0)[0]

mp.plot(v, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4306930…

In [25]:
# create instance of Deformer
deformer_ = Deformer(v, f, handle_indices)

# step 1: Smooth the original mesh (except vertex handles)
B = deformer_.smooth_mesh(v)

mp.plot(B, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.4313118…

#### Step 2: Deforming the smooth mesh

This is the `B_prime= deform_ops.smooth_mesh(v,f,handle_indices,v_new)`

As before, `smooth_mesh`, takes in the original vertices and faces, the indices of the handles, and a value for the `new` positions of the vertices. Unlike the previous step, we now plug in `v_new` for the last parameter. 

The new deformed mesh is computed similarly to the previous step, by solving the minimization:
<img align="center" width="200" src="https://i.imgur.com/xv8ZcsA.png">
<!-- $$ 
\begin{aligned} \min_v& \quad v^T \textbf{L}_\omega \textbf{M}^{-1} \textbf{L}_\omega \textbf{v} \\
 \text{subject to}&
 \quad \textbf{v}_H = t(\textbf{o}_H),
\end{aligned}
$$ -->
where $t(\textbf{o}_H)$ are the new handle vertex positions after applying the user's transformation. We call this mesh $\mathcal{B}'$.

*Relevant `scipy` functions:* `scipy.sparse.linalg.spsolve` 

In [26]:
# compute new positions for one of the handles 
new_positions = np.copy(v)
handle_choice = 2 # choose a handle to move
handle_idx = np.where(labels == handle_choice)[0] # indices of handle # handle choice
new_positions[handle_idx] = new_positions[handle_idx] + np.array([0.1, 0.5, 0.5]) # move the handle by 0.5 in each direction
# # step 2: find the smoothed mesh for the target mesh positions 
B_prime = deformer_.smooth_mesh(new_positions)

mp.plot(B_prime, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5313118…

#### Step 3: Encode Displacement Vectors (in local frame) 

<!-- <img align="left" width="200" src="img/hand_bd.png"> 
<img align="left" width="200" src="img/hand_td.png">
<br clear="both"/>
<img align="left" width="200" src="img/woody_bd.png">
<img align="left" width="200" src="img/woody_td.png">
<br clear="both"/> -->

<!-- *Fig 4: Displacements on $B$ (left) and $B'$ (right)* -->

This is done in two steps :

* (1) `coeffs, tangent_vectors = deformer_.compute_detail_encoding(B)` : which finds a reference frame for B by using:

  * $n_i$: the normal at $v$

  * $x_i$: normalized projection of one of $v$'s outgoing edges onto the tangent, and

  * $y_i$: the cross product of the normal and the tangent projection.

  These define a detail, $d$, as a set of coefficients \[d1,d2,d3\] multiplied by \[$x_i$, $y_i$, $n_i$\]. We save the coefficents to the `coeffs` array. We further save the values of all the $x_i$s to use in the computation of the reference frame for $B'$.

* (2) `dets =  deformer_.apply_detail_encoding(B_prime,coeffs, tangent_vectors`: which returns the details we need to add back to reconstruct the deformed surface using:

  * $n_i'$: the normal at $v$ in `B_prime`

  * $x_i'$: the projection we obtained in step (1) which is saved in tangent_vectors

  * $y_i'$: the cross product of the normal of $B'$ and the tangent projection.

  We then compute the detail in $B'$ as each coefficient (saved in `coeffs`) multiplied by each component $x_i, y_i, n_i$. 

  These details are then added to $B'$ to give us our final surface $S'$. 

___ 
The high-frequency details on the original surface are extracted from $\mathcal{S}$ and transferred to $\mathcal{B}'$. We first encode the high-frequency details of $\mathcal{S}$ as displacements w.r.t. $\mathcal{B}$.
We define an orthogonal reference frame on every vertex $v$ of $\mathcal{B}$ using:
1. The unit vertex normal
2. The normalized projection of one of $v$'s outgoing edges onto the tangent plane defined by the vertex normal. A stable choice is the edge whose projection onto the tangent plane is longest.
3. The cross-product between (1) and (2)

For every vertex $v$, we compute the displacement vector that takes $v$ from $\mathcal{B}$ to $\mathcal{S}$ and represent it as a vector in $v$'s reference frame. 
For every vertex of $\mathcal{B}'$, we also construct a reference frame using the normal and the SAME outgoing edge we selected for $\mathcal{B}$ (not the longest in $\mathcal{B}'$; it is important that the edges used to build both reference frames are the same). We can now use the displacement vector components computed in the previous paragraph to define transferred displacement vectors in the new reference frames of $\mathcal{B}'$. See Figure 4 for an example.
Applying the transferred displacements to the vertices of $\mathcal{B}'$ generates the final deformed mesh $\mathcal{S}'$. See Figure 5 for an example.

<!-- <img align="left" width="200" src="img/hand_f.png">
<img align="left" width="200" src="img/woody_f.png">
<br clear="both"/> -->

*Fig 5: Final Deformation Results*

Recommended outputs:
- Provide screenshots for 4 different deformed meshes. For each example, provide a rendering of $\mathcal{S}$, $\mathcal{B}$, $\mathcal{B}'$ and $\mathcal{S}'$.


In [27]:
# step 3.1: encoding the details of B
coeffs, tangents = deformer_.compute_detail_encoding(B)
# step 3.2: compute the details of B_prime and add them back 
B_prime_dets = deformer_.apply_detail_encoding(B_prime, coeffs, tangents)
S_prime = B_prime + B_prime_dets

mp.plot(S_prime, f, c=labels)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.5306930…

## Putting it all together: deforming 4 meshes

If you want to check specific quantities for each mesh, you can change the handle_choice or shift_choice in the array. Just ensure that the values you choose are all consistent in indicies (i.e. the shift value, handle value, label_file and mesh_file are all at the same index within their respective arrays.)

In [34]:
mesh_files = ['data/hand.off', 'data/woody-hi.off', 'data/cactus.off', 'data/bumpy_plane.off']
labels_files = ['data/hand.label.npy', 'data/woody-hi.label.npy', 'data/cactus.label.npy', 'data/bumpy_plane.label.npy']
handle_choice = [2,3,2,1]
shift_choice = [[0,0.5,0], [-0.3,0,0], [0,0.5,0], [0,0,1]]

In [29]:
def load_and_deform_mesh(mesh_file, labels_file, handle_choice, shift_choice):
    v, f = igl.read_triangle_mesh(mesh_file)
    labels = np.load(labels_file).astype(int)
    v -= v.min(axis=0)
    v /= v.max()
    new_pos = v.copy()
    handle_indices = np.where(labels!=0)[0]
    handle1_idx = np.where(labels == handle_choice)[0] # indices of handle vertices
    new_pos[handle1_idx] = v[handle1_idx]+np.array(shift_choice)
    deformer_ = Deformer(v, f, handle_indices)
    # smooth original mesh (except vertex handles)
    B = deformer_.smooth_mesh(v)
    # smooth the target mesh (except vertex handles)
    B_prime =  deformer_.smooth_mesh(new_pos)
    # encode the details of B into a set of coefficients and tangents
    coeffs, tangents = deformer_.compute_detail_encoding(B)
    # apply the detail encoding to the target mesh
    B_prime_dets = deformer_.apply_detail_encoding(B_prime,coeffs, tangents)
    S_prime = B_prime + B_prime_dets
    return (labels, f, v, B, B_prime, S_prime)

def plot_mesh(mesh_file, labels_file, handle_choice, shift_choice, one_row=True, wireframe=True):
    labels, f, v, B, B_prime, S_prime = load_and_deform_mesh(mesh_file, labels_file, handle_choice, shift_choice)
    if (one_row):
        p1 = mp.subplot(v, f, c=labels, s=[1,4,0], shading={"wireframe": wireframe})
        p2 = mp.subplot(B, f, c=labels, data=p1, s=[1,4,1], shading={"wireframe": wireframe})
        p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[1,4,2], shading={"wireframe": wireframe})
        p4 = mp.subplot(S_prime, f, c=labels, data=p1, s=[1,4,3], shading={"wireframe": wireframe})
    else:
        p1 = mp.subplot(v, f, c=labels, s=[2,2,0], shading={"wireframe": wireframe})
        p2 = mp.subplot(B, f, c=labels, data=p1, s=[2,2,1], shading={"wireframe": wireframe})
        p3 = mp.subplot(B_prime, f, c=labels, data=p1, s=[2,2,2], shading={"wireframe": wireframe})
        p4 = mp.subplot(S_prime, f, c=labels, data=p1, s=[2,2,3], shading={"wireframe": wireframe})

### Plot of hand: 
Plotted in a 2x2 for space constraints, but tovisualize in one line, change `one_row` to `True`

In [30]:
mesh_index = 0
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False, wireframe = False)

### Plot of woody: 
Plotted in a 2x2 for space constraints, but tovisualize in one line, change `one_row` to `True`

In [31]:
mesh_index = 1
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False)

### Plot of cactus: 
Plotted in a 2x2 for space constraints, but tovisualize in one line, change `one_row` to `True`

In [32]:
mesh_index = 2
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False)

### Plot of bumpy plane: 
Plotted in a 2x2 for space constraints, but tovisualize in one line, change `one_row` to `True`

Note that for this mesh, I chose to hide the wireframe. This is because there are very many triangles, amking it difficult to see the colors of the mesh.

In [35]:
mesh_index = 3
plot_mesh(mesh_files[mesh_index], labels_files[mesh_index], handle_choice[mesh_index], shift_choice[mesh_index], one_row = False, wireframe=False)

## Extra-credit: 

Along with this notebook file, I am submtiting two files: `deform_ops.py` and `deform_ops_chol.py`. These files each hold a version of the operations I used to do the deformations (i.e. smooth mesh and encode details). One of these classes is imported in the beginning, and an instance is created in the interactive function in the line where I declare `deformer_ = Deformer(...)`.  `deform_ops.py` holds the operations before I optimized (using sparse cholesky and other vectorizations), and `deform_ops_chol.py` contains the optimized operations for performance. The way the code is structured, it functions with either file imported. In order to switch between optimized and nnon-optimized, it suffices to just comment out the import you do not wish to use.  




__ 

To achieve real-time performance, you must prefactor the sparse bi-Laplacian matrix appearing in both linear systems. After the user specifies vertex sets $H$ and $F$, you can factorize the matrix $L_\omega M^{-1} L_\omega$ (using a Cholesky $LL^T$ factorization) and then re-use the factorization to solve both linear systems efficiently. This is an optional part of the exercise; if your implementation does not achieve interactive frame-rates (10+ fps) on the gingerbread mesh, it will not receive the full score. This might require additional vectorizations.

*Available Packages*: `scikit-sparse`, `numba`.
*Relevant functions*: `sksparse.cholmod`, `numba.jit`, `numpy.einsum`, `scipy.sparse.linalg.splu`